<strong><center><h1>Analyzing Stock Options with Machine Learning</h1></center></strong>

## **1. Introduction**

#### For many years, traders and merchants developed and practiced techniques involving the prediction of prices for goods. These techniques were then used for predicting stock prices (Hayes, 2019). Many of these techniques are still used today. Recently, more traders have been relying on the use of algorithms for automated trading and price prediction. An advantage many have found with using algorithms is that it strips trading of personal attachment (i.e. emotions). This project in focuses more on the prediction of stock options.

#### Stock options are contracts that give an individual the right to **buy** or **sell** a stock at a specific price by a specific date (Palmer, 2019). They can either be a buy or a sell contract known as a **call** and a **put**, respectively. These options allow an individual to buy or sell their option if the price of a stock reaches a certain price point. This project involves the use of historical stock option data to predict a call or a put for future option investments.

#### The goal of this project is to make observations of multiple models involved with predicting stock options and compare them.

## **2. Data**

### **A. Importing Libraries and Data**

In [1]:
from copy import deepcopy as copy
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from numpy import shape
from math import log, sqrt, exp, pi
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import animation

import tensorflow as tf
from tensorflow import keras 
from copy import deepcopy as copy
import IPython.display as ipd 
import importlib
import partitionlib as ptl

KeyboardInterrupt: 

#### There are multiple sources of data located in the **Sample_L2_2018** directory. The name of the dataset correlates to when the data was collected (i.e. **20180102** = **01/02/2018**).


#### For the initial experiments, we will use the data for the first month. 

In [ ]:
options = pd.concat(map(pd.read_csv, glob.glob("Sample_L2_2018/L2_options_2018*.csv")))
options

#### In order to help determine relevant features, we will call the <code>unique()</code> method provided by the pandas library in order to check unique values in a feature.

#### In this case, the *OptionExt* feature only contains a value of *NaN*. Therefore, we can exclude it in our data.

In [ ]:
options['OptionExt'].unique()

In [ ]:
options['Exchange'].unique()

In [ ]:
options['Expiration'].unique()

In [ ]:
options['OpenInterest'].unique()

#### The *Type* feature in our data contains strings to classify the data. We will convert the data into integers by using the <code>astype()</code> method and passing *int* as an argument.

In [ ]:
options['Type'] = (options['Type'] != 'call').astype(int)

#### Now we can represent the *Type* feature as integers and drop the irrelevant features from our data.

In [ ]:
X = options.drop(['DataDate', 'OptionExt', 'AKA', 'OptionSymbol', 'Exchange'], axis=1).copy()

In [ ]:
X

### **B. Explanation of Features**

&emsp; *provided by Historical Option Data: https://www.historicaloptiondata.com/content/historical-options-data-file-structures-0*
* #### **Expiration:** This is the time it takes until the stock option expires. After the contract expires, the buyer must commit to their option (call or put).
* #### **Strike:** The price at which an option holder can buy or sell.
* #### **Last:** The last traded price of the option.
* #### **Bid:** The price an option buyer is willing to pay for the contract.
* #### **Ask:** The price an option seller is willing to sell for the contract.
* #### **Volume:** The number of contracts traded.
* #### **OpenInterest:** Always a day behind - the number of options that are held by traders and investors in active positions.
* #### **Implied Volatility(IV):** A measure of the estimate of how much the price could change. It relates to how likely the traders believe the option can make a change.
* #### **Delta:** A measure of how much the option price would change in relation to the underlying stock price. Delta = .50 -> option changes 50 cents for every 1 dollar the stock moves.
* #### **Gamma:** A measure of how fast the Delta will change when the stock price changes. A high number means this is a very explosive option and can gain or lose value quickly.
* #### **Theta:** A measure of how fast the option is losing value per day due to time decay. As the expiration day arrives, the theta increases.
* #### **Vega:** A meaasure of how sensitive the option price is to a change in the implied volatility. Options that have a long time until expiration are more sensitive to change in IV.

### **C. Preprocessing the Data**

#### Here, we converted the date into the days until expiration with the <code>to_datetime()</code> method.

In [ ]:
X['Expiration'] = pd.to_datetime(X['Expiration']).sub(pd.Timestamp('01/02/2018')).dt.days
X

#### In order to stay consistent with the other features, we must convert the time until expiration into units of years.

In [ ]:
X['Expiration'] = X['Expiration'].apply(lambda x: x / 365)
X

## **3. The Black-Scholes Model**

#### The Black-Scholes Model, or Black-Scholes-Merton (BSM) Model, is one of the most well-known models used in finance. The model implies that the most relevant features of pricing a stock option depends on:
#### &emsp;&emsp;$S$: Stock Price
#### &emsp;&emsp;$X$: Exercise Price
#### &emsp;&emsp;$r$: Risk-Free Interest Rate
#### &emsp;&emsp;$T$: Time until Expiration
#### &emsp;&emsp;$\theta$: Standard Deviation of *log* returns (volatility)<br><br>

<center><h4>The BSM involves a mathematical formula that determines a fair price for stock options.</h4></center><br>

<center><h4>$C_0$ $=$ $S_0N(d_1) - Xe^{(-rt)}N(d_2)$</h4></center><br>

<center><h4>where</h4></center><br>

<center><h4>$d_1 = \frac{ln\frac{S_t}{X}+(r+\frac{\theta^2}{2}T}{\theta\sqrt{T}}$, $d_2 = \frac{ln\frac{S_t}{X}+(r-\frac{\theta^2}{2}T}{\theta\sqrt{T}}$</h4></center><br>

<center><h4>$N(X)$ gives us the probability where our random variable will be less than or equal to $X$.</h4></center><br>

<center><h4>$0 < N(X) < 1$</h4></center><br>

<center><h4>The price of the stock, $S_0$, is weighted by the probability $N(X)$:</h4></center><br>

<center><h4>$S_0N(d_1)$</h4></center><br>

<center><h4>The amount you would pay is determined by multiplying $X$ and $e$ raised to $(-rt)$, then weighted by $N(X)$:</h4></center><br>

<center><h4>$Xe^{(-rt)}N(d_2)$</h4></center>

### **A. Implementation**

#### This implementation of the BSM was provided by QuantConnect.

In [ ]:
class BsmModel:
    
    def __init__(self, option_type, price, strike, interest_rate, expiry, volatility, dividend_yield=0):
        self.type = option_type # 0 for call, 1 for put
        self.s = price # underlying asset price
        self.k = strike # strike price of option
        self.r = interest_rate # risk-free interest rate
        self.T = expiry # time till expiration in years
        self.sigma = volatility # standard deviation of log returns
        self.q = dividend_yield # dividend continuous rate
        
    def n(self, d):
        # cumulative probabilty distribution function of standard normal distribution; N(X)
        return norm.cdf(d)
    
    def dn(self, d):
        # first order derivative of n(d)
        return norm.pdf(d)
    
    def d1(self):
        d1 = (log(self.s / self.k) + (self.r - self.q + self.sigma ** 2 * 0.5) * self.T) / (self.sigma * sqrt(self.T))
        return d1
    
    def d2(self):
        d2 = (log(self.s / self.k) + (self.r - self.q - self.sigma ** 2 * 0.5) * self.T) / (self.sigma * sqrt(self.T))
        return d2
    
    def bsm_price(self):
        d1 = self.d1()
        d2 = d1 - self.sigma * sqrt(self.T)
        if self.type == 0:
            price = exp(-self.r*self.T) * (self.s * exp((self.r - self.q)*self.T) * self.n(d1) - self.k * self.n(d2))
            return price
        elif self.type == 1:
            price = exp(-self.r*self.T) * (self.k * self.n(-d2) - (self.s * exp((self.r - self.q)*self.T) * self.n(-d1)))
            return price
        else:
            print("option either 0 (call) or 1 (put)")

In [ ]:
bsmX = X[['Type', 'UnderlyingPrice', 'Strike', 'Expiration', 'IV']]
bsmX

In [ ]:
bsmX['BSM'] = bsmX.apply(lambda x: BsmModel(x["Type"], x["UnderlyingPrice"], x["Strike"], 0.1, x["Expiration"], x["IV"]).bsm_price(), axis=1)

In [ ]:
bsmX

## **4. The Greek Letters**

#### In our dataset, there are 4 greek letters that were provided: *Delta, Gamma, Theta*, and *Vega*. Each greek letter is a parameter that is used to measure different dimensions in options.
* #### Delta: The measure of how much the option price would change in relation to the underlying stock price.
* #### Gamma: The measure of how fast the Delta will change when the stock prices change.
* #### Theta: The measure of how fast the option is losing value per day due to time decay.
* #### Vega: The measure of how sensitive the option price is to a change in the implied volatility.
* #### Rho: The measure of how sensitive the option price is to a change in interest rate.

#### These values are typically calculated on a daily basis. Once the values for these letters have been determined, traders would apply them to the BSM model to asses option prices.

### **A. Implementation**

#### Since our dataset has already provided the values for the greeks (*Delta, Gamma, Theta, and Vega*), we will use them to assess the BSM model created in the previous section.

In [ ]:
delta = X[['Delta']].to_numpy()
gamma = X[['Gamma']].to_numpy()
theta = X[['Theta']].to_numpy()
vega = X[['Vega']].to_numpy()

In [ ]:
price = bsmX.iloc[:, 1].to_numpy()
time = bsmX.iloc[:, 3].to_numpy()

In [ ]:
fig = plt.figure(figsize=(20,11))
ax = plt.axes(projection='3d')
ax.set_xlabel('Stock Price')
ax.set_ylabel('Time to Expiration')
ax.set_zlabel('Gamma')
ax.scatter(price, time, gamma, color='r')
plt.show()

## Tensorflow

In [ ]:
X = pd.read_csv("options.csv", error_bad_lines=False)
X = X.drop(columns=['OptionExt','OptionSymbol','AKA','OptionSymbol','Exchange','Close'])
X['Expiration'] = pd.to_datetime(X['Expiration']).sub(pd.Timestamp('01/05/2018')).dt.days
X['DataDate'] = pd.to_datetime(X['DataDate']).sub(pd.Timestamp('01/02/2018')).dt.days
X['Type'] = (X['Type'] != 'call').astype(int)
K = pd.get_dummies(X['UnderlyingSymbol'])
dic = {'AMZN': 0, 'F':1, 'FXI':2,'GE': 3, 'MSFT':4, 'NFLX':5 ,'QCOM': 6, 'UVXY': 7,'VIX': 8, 'XLF': 9 }
X['UnderlyingSymbol'] = X['UnderlyingSymbol'].apply(lambda x: dic[x]) 
X = X.dropna()
T = X['Open'].copy()
X = X.drop(columns=['Open'])
X

In [ ]:
T = pd.DataFrame(T)
T

In [ ]:
tf.keras.backend.set_floatx('float64')
nEpochs = 500
bSize = 1000
def partition_data(X, T):
    """Partition data into train and test splits."""
    data, targets = ptl.partition(copy(X.values), copy(T.values))
    Xtrain, Xtest = data
    Ttrain, Ttest = targets
    
    return Xtrain, Xtest, Ttrain, Ttest
def standardize(X, mean, std):
    return (X-mean) / std
def plot_fit(Y, T, title=None):
    """ Plot the passed predictions and targets while also computing
        and printing the RMSE.
    """
    plt.plot(T, label="target")
    plt.plot(Y, linewidth=3, label="prediction")
    plt.legend()
    plt.ylabel("SO2")
    plt.xlabel("Samples")
    plt.title(title)
    # RMSE
    rmse = np.sqrt(np.mean((Y - T)**2))
    print("RMSE: {}".format(rmse))

In [ ]:
Xtrain, Xtest, Ttrain, Ttest = partition_data(X, T)
Xtrain =  np.array(Xtrain, dtype=np.float)
Xtest =  np.array(Xtest, dtype=np.float)
Ttrain =  np.array(Ttrain, dtype=np.float)
Ttest =  np.array(Ttest, dtype=np.float)
N, D = X.shape
Ntrain, Ntest = Xtrain.shape[0], Xtest.shape[0]
print("Train data shape: {}".format(Xtrain.shape))
print("Train target shape: {}".format(Ttrain.shape))
print("Test data shape: {}".format(Xtest.shape))
print("Test target shape: {}".format(Ttest.shape))
print("Number of total samples: {}".format(N))
print("Number of total features: {}".format(D))
print("Number of training samples: {}".format(Ntrain))
print("Number of testing samples: {}".format(Ntest))

#Xtrain = Xtrain.reshape([Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[2], -1])
#Xtest = Xtest.reshape([Xtest.shape[0], Xtest.shape[1], Xtest.shape[2], -1])

pd.DataFrame(Xtrain).describe()

In [ ]:
n1 = 500
n2 = 800
n3 = 1000
n4 = 800
n5 = 500
n6 = 480

openRateModel = keras.Sequential([
    keras.layers.Input(shape=D,),
    keras.layers.Dense(units=n1, activation='relu'),
    keras.layers.Dense(units=n1, activation='relu'),
    keras.layers.Dense(units=n2, activation='relu'),
    keras.layers.Dense(units=n2, activation='relu'),
    keras.layers.Dense(units=n3, activation='relu'),
    keras.layers.Dense(units=n4, activation='relu'),
    keras.layers.Dense(units=n4, activation='relu'),
    keras.layers.Dense(units=n5, activation='relu'),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(units=n6),
    keras.layers.Dense(1)
])

openRateModel.summary()

In [ ]:
mean = np.mean(Xtrain, axis=0)
std = np.std(Xtrain, axis=0)
XtrainS = standardize(Xtrain, mean, std)
XtestS = standardize(Xtest, mean, std)
pd.DataFrame(XtrainS).describe()

In [ ]:
openRateModel.compile(loss="mse", optimizer="nadam", metrics=["mse","mae"])

In [ ]:
history = openRateModel.fit(XtrainS, Ttrain, epochs=nEpochs, validation_split=0.2, batch_size=bSize)

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel("# itrations")
plt.ylabel("loss")

In [ ]:
Ytest_probs = openRateModel.predict(Xtest)
plot_fit(Ytest_probs, Ttest)